1. Use the “Classic Outlook” (needed for win32com)
    win32 works with classic outlook and it needs microsoft 365 subscription
    
2. If you stick with Microsoft Store Outlook
   You cannot use win32com — because it has no COM interface.

In [1]:
# pip install pywin32

In [1]:
# outlook.bat is used for adding outlook.exe to system path, this is not used

### import Libs

In [35]:
import win32com.client as win32
import os
import time
import pandas as pd
import logging

### Variables

In [13]:
MailBoxName = '@gmail.com'
ToRecipients = '@gmail.com'
ReplyRecipients = '@gmail.com'
AttachmentFilePath = '"C:\Users\UserName\OneDrive\Desktop\My Work Experience\intelligent-automation-python\README.md"'

### Connect to Outlook

In [5]:
## for checking 64 bit or 32 bit 
# import platform
# print(platform.architecture())

In [7]:
# os.startfile('outlook')

In [26]:
outlook = win32.Dispatch("Outlook.Application")

### Send Email through Outlook

In [27]:
name_space = outlook.GetNamespace("MAPI") 

In [28]:
# selecting the profile
profile = name_space.Folders[MailBoxName]

In [29]:
# selecting the Sender Account
SenderAccount = outlook.Session.Accounts[MailBoxName]

In [32]:
email = outlook.CreateItem(0)

In [34]:
body_html = '''<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8" />
  <meta name="viewport" content="width=device-width,initial-scale=1" />
  <title>Message</title>
  <style>
    body { font-family: Arial, Helvetica, sans-serif; font-size: 14px; color: #111; }
    .container { max-width: 680px; margin: 20px auto; padding: 16px; }
  </style>
</head>
<body>
  <div class="container">
    <p>Hi,</p>

    <p>PFA,</p>

    <p>Thanks and Regards,</p>
  </div>
</body>
</html>
'''

In [33]:
email.To = ToRecipients
email.ReplyRecipients.Add(ReplyRecipients)
email.Subject = 'Test Subject'
email.HTMLBody = body_html   # HTML Format
# email.Body = MailBody        # normal Format
email.Attachments.Add(AttachmentFilePath)
email.Save()
time.sleep(2)

In [ ]:
email.Send()

### Save Outlook email

In [ ]:
# initializing logging

# Configure logging
logging.basicConfig(
    level=logging.INFO,                      # Set minimum log level
    format="%(asctime)s - %(levelname)s - %(message)s",  # Log format
    handlers=[
        logging.StreamHandler()              # Output logs to console
    ]
)

###### Variables

In [4]:
SaveEmailFolderPath  = 'C:/Users/Downloads'
MailBoxName = '@gmail.com'

##### Main code

In [ ]:
name_space = outlook.GetNamespace("MAPI") 

In [ ]:
# selecting the profile
profile = name_space.Folders[MailBoxName]

In [ ]:
# Connect to Sent Items Folder, SavedSentItems Folder
SentItemsFolder = name_space.Folders[MailBoxName].Folders["Sent Items"]
SavedSentItemsFolder = name_space.Folders[MailBoxName].Folders["Saved Sent Items"]

In [ ]:
# Filtering based on Subject
Emails = SentItemsFolder.Items.Restrict("@SQL=""urn:schemas:httpmail:subject"" LIKE '{}'".format('DealerShip'))

In [ ]:
# get Emails Count
EmailCount = Emails.Count

In [ ]:
# Sorting Emails for fecthing latest email
Emails.sort("[ReceivedTime]", False)

In [5]:
def Extract_Reg_Number(text):
    matches = re.findall(r"\b\d{9}\b", text)
    if len(matches)!=0:
        for i in matches[0]:
            if i!='':
                return i

In [ ]:
# Looping thorugh each email
if EmailCount > 0:
    for i in range(EmailCount, 0, -1):
        email = Emails[i-1]
        # access Email Sent Time
        SentTime = pd.to_datetime(str(email.ReceivedTime))
        email_Subject = email.Subject
        # Extracting 9 digit Registration Number from Email Subject
        Reg_No = Extract_Reg_Number(email_Subject)
        if Reg_No != '':
            logging.info("Extracted Registration Number from Email Subject")
        else:
            logging.warning(f"Extraction Registration Number from Email Subject-{email_Subject} not Success")
            continue
#         if os.path.exists(SaveEmailFolderPath):
        output_dir = f"{SaveEmailFolderPath}/{Reg_No}"
        output_dir.mkdir(parents=True, exist_ok=True)
        output_file_path = f"{output_dir}/Email.msg"
        email.SaveAs(output_file_path)
        logging.info("Email Saved Successfully to Output Folder")
        email.move(SavedSentItemsFolder)
        logging.info("Email Moved Successfully to Processed Folder")
            

In [ ]:
# logging.debug("This is a debug message")     # Will not show (below INFO level)
# logging.info("This is an info message")
# logging.warning("This is a warning")
# logging.error("This is an error message")
# logging.critical("This is a critical message")